# Importing libraries

In [1]:
import numpy as np
from scipy.optimize import linprog, minimize_scalar
from numpy.linalg import norm, inv
from numdifftools import Gradient

In [2]:
def f(x):
    return x[0] ** 2 + 3 * x[0] * x[1] + 4 * x[1] ** 2

A = np.array([1, 1], dtype=float)
b = [1]

# Reduced Gradient

In [3]:
def reduced_gradient(f, A, b, x0, t):
    x = x0
    n = len(x)
    m = len(b)
    B = [*range(0, m)]
    R = [*range(m, n)]

    while True:
        try:
            grad = -np.dot(np.dot(inv(A[B]), A[R]).T, Gradient(f)(x)[B]) + Gradient(f)(x)[R]
        except:
            grad = -np.dot(np.dot(A[B], A[R]).T, Gradient(f)(x)[B]) + Gradient(f)(x)[R]

        J = []
        k = 0
        for j in R:
            if grad[k] < 0 or x[j] > 0:
                J.append(k)
            k = k + 1
        
        d_R = [d if list(grad).index(d) in J else 0 for d in grad]
        
        alpha_R = np.inf
        k = 0
        for j in R:
            if grad[k] > 0 and x[j] > 0:
                alpha = x[j] / grad[k]
                alpha_R = min(alpha_R, alpha)
            k = k + 1

        try:
            d_B = -p.dot(np.dot(inv(A[B]), A[R]), d_R)
        except:
            d_B = -np.dot(np.dot(A[B], A[R]), d_R)

        d = np.array(list(d_B) + list(d_R), dtype=float)

        alpha_B = np.inf
        for j in B:
            if d[j] < 0:
                alpha = x[j] / (-d[j])
                alpha_B = min(alpha_B, alpha)

        alpha_bar = min(alpha_B, alpha_R)
        phi = lambda alpha: f(x + alpha * d)
        alpha = minimize_scalar(phi, bounds=(0, alpha_bar), method='bounded').x

        x = x + alpha * d

        if abs(alpha - alpha_B) <= t:
            index_r = -1
            for i in B:
                if x[i] == 0:
                    index_r = i
                    break
            
            index_b = -1
            for j in R:
                if x[j] > 0:
                    index_b = j
                    break
            
            if index_r != -1 and index_b != -1:
                B[B.index(index_r)] = index_b
                R[R.index(index_b)] = index_r
            else:
                return x
        
        if norm(d_B) <= t:
            return x

In [4]:
x0 =  [0, 1]
x_sol = reduced_gradient(f, A, b, x0, 0.0001)
print(x_sol)

[9.99975825e-01 2.41748506e-05]
